# Базовые слои

В документе рассматривается вопрос об устойчивости базовых карт. Сами карты были созданы с использованием медианы зимних снимков и [импортированы в проект](000_Initialization.ipynb#import_landsat_composites).

## Устойчивость базовых слоев

Прежде, чем начинать работать с базовыми слоями, нужно понять, насколько они отличаются друг от друга. В идеале, нужно, чтобы слои, полученные в разные годы, были одинаковыми, а различались в них лишь те участки, которые подверглись реальному изменению. В данном подразделе рассматривается вопрос о том какова величина этих различий.

In [ ]:
import os

import utilites
reload(utilites)

from utilites import (
    get_gisbase_path,
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)

print get_location_path()

from grasslib import GRASS

In [ ]:
grs = GRASS(gisbase=get_gisbase_path(), 
            dbase=get_grassdata_path(), 
            location=get_location_name(),
            mapset='basemaps'
)
# print grs.grass.read_command('g.mapset', mapset='basemaps')
print grs.grass.read_command('g.mapset', flags='p')

In [ ]:
%tb
# Установим охват, чтобы он накрывал все сцены
grs.grass.run_command('g.region', region='all_scenes@landsat')

grs.grass.run_command('g.remove', type='rast', pattern='diff*', flags='f')

for band_num in range(1, 12):
    print band_num
    
    for (new_year_pref, old_year_pref, diff_pref) in [
        ('composite16.', 'composite15.', 'diff1615.'),
        ('composite17.', 'composite16.', 'diff1716.')
    ]:
    
        new_year = new_year_pref + str(band_num) + '@basemaps'
        old_year = old_year_pref + str(band_num) + '@basemaps'
        diff = diff_pref + str(band_num)
        command = "%(out)s = %(new_year)s - %(old_year)s" % dict(out=diff, new_year=new_year, old_year=old_year)
        print command
        grs.grass.mapcalc("${out} = ${new_year} - ${old_year}", 
                          out=diff, new_year=new_year, old_year=old_year
                         )        
        norm_diff = 'norm.' +  diff
        grs.grass.mapcalc("${out} = (${new_year} - ${old_year}) / (${new_year} + ${old_year})", 
                          out=norm_diff, new_year=new_year, old_year=old_year)

In [ ]:
for band_num in range(1, 12):
    print "----- BAND %s ------- " % (band_num, )
    print grs.grass.read_command('r.report', map='diff1615.'+str(band_num), 
                                 units='c', nsteps=10, sort='desc', flags='h')
    print grs.grass.read_command('r.report', map='diff1716.'+str(band_num), 
                                 units='c', nsteps=10, sort='desc', flags='h')

Итак, видно, что мода почти во всех каналах приходится приблизительно на диапазон от -0.05 до 0.15 (для разности 16-го года и 15-го) и от -0.1 до 0.15 для разности 17-го года и 16-го. В температурных каналах это различие, понятное дело, больше. Вообще, это довольно большая разность, особенно если учесть, что сами данные лежат в диапазоне от нуля до единицы, а основной массив плотности гистограммы приходится на диапазон (0.1, 0.4).
Правда, есть и хорошая новость: основная часть больших различий приходится на территории, которые не покрыты лесами. Внутри залесенных участков разносит в обычно меньше.

**TODO:** положить картинки разностей для наглядности.

In [ ]:
# grs.rast.RasterRow('diff1615.1')

## Нормализация базовых слоев относительно друг друга

Посмотрим, насколько эти базовые слои "бъются" между собой при использовании процедуры нормализации, описанной в [соответствующем документе](007_Normalization.ipynb). Для этого найдем псевдоинвариантные пиксели для разных пар растров и посчитаем относительно этих пикселей нормирующие коэффициенты. Степень близости полученных коэффициентов будет служить оценкой схожести базовых слоев на неизменившихся участках.

Импортируем функции нормализации, описанные в указанном документе.

In [ ]:
import normalization
reload(normalization)

from normalization import get_norm_coef, find_pif

Возмем для примера 5-й канал и рассчитаем нормирующие коэффициенты для каждой пары растров:

In [ ]:
REGR_EPS = 0.35
MAXITER = 50

grs.grass.run_command('g.region', region='all_scenes@landsat', res=120)

params1615 = find_pif(grs.grass, 'composite16.5', 'composite15.5', 
                      'tmp.PIF.1615', regression_eps=REGR_EPS, max_iteration=MAXITER)
print params1615

params1716 = find_pif(grs.grass, 'composite17.5', 'composite16.5', 
                      'tmp.PIF.1716', regression_eps=REGR_EPS, max_iteration=MAXITER)
print params1716


params1715 = find_pif(grs.grass, 'composite17.5', 'composite15.5', 
                      'tmp.PIF.1715', regression_eps=REGR_EPS, max_iteration=MAXITER)
print params1715

Нормирующие коэффициенты для 15-го года:

In [ ]:
print get_norm_coef(grs.grass, 'composite15.5', 'tmp.PIF.1615')
print get_norm_coef(grs.grass, 'composite15.5', 'tmp.PIF.1715')

{'intercept': -93.68254592043874, 'factor': 472.2852372468748}
{'intercept': -7.6259174283960025, 'factor': 34.73406255368474}

Нормирующие коэффициенты для 16-го года:

In [ ]:
print get_norm_coef(grs.grass, 'composite16.5', 'tmp.PIF.1615')
print get_norm_coef(grs.grass, 'composite16.5', 'tmp.PIF.1716')

Нормирующие коэффициенты для 17-го года:

In [ ]:
print get_norm_coef(grs.grass, 'composite17.5', 'tmp.PIF.1715')
print get_norm_coef(grs.grass, 'composite17.5', 'tmp.PIF.1716')